CNN

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf


from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

# 데이터셋 다운로드 오류 (링크 깨짐 해결)
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

dataset_name = 'cats_vs_dogs'


# 처음 80%의 데이터만 사용
train_dataset = tfds.load(name=dataset_name, split='train[:80%]')

# 최근 20%의 데이터만 사용
valid_dataset = tfds.load(name=dataset_name, split='train[80%:]')

def preprocess(data):
    # x, y 데이터를 정의
    x = data['image']
    y = data['label']
    # image 정규화(Normalization)
    x = x / 255
    # 사이즈를 (224, 224)로 변환
    x = tf.image.resize(x, size=(224, 224))
    # x, y  데이터를 return 
    return x, y

def solution_model():
    train_data = train_dataset.map(preprocess).batch(32)
    valid_data = valid_dataset.map(preprocess).batch(32)

    transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    transfer_model.trainable=False


    model = Sequential([
        transfer_model, #trainsfer learning 부분
        Dropout(0.5),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax'),])
    

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

    checkpoint_path = "tmp_checkpoint.ckpt"

    checkpoint = ModelCheckpoint(checkpoint_path,
                                 save_weights_only=True,
                                 save_best_only=True,
                                 monitor='val_loss',
                                 verbose=1)

    model.fit(train_data,
              validation_data=valid_data,
              epochs=15,
              verbose=1,
              callbacks=[checkpoint],
              )

    model.load_weights(checkpoint_path)


    return model


if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-cats-vs-dogs_1.h5")


In [ ]:
import urllib
import zipfile

import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout,Flatten, Dense, Conv2D, MaxPooling2D,Reshape
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential


def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/germantrafficsigns.zip'
    urllib.request.urlretrieve(url, 'germantrafficsigns.zip')
    with zipfile.ZipFile('germantrafficsigns.zip', 'r') as zip_ref:
        zip_ref.extractall()

def preprocess(image, label):
    image = image/255
    return image, label


def solution_model():
    download_and_extract_data()
    BATCH_SIZE = 32
    IMG_SIZE = 30

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='train/',
        label_mode='categorical',
        image_size=(IMG_SIZE,IMG_SIZE),
        batch_size = BATCH_SIZE)

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='validation/',
        label_mode='categorical',
        image_size=(IMG_SIZE,IMG_SIZE),
        batch_size = BATCH_SIZE)

    train_ds = train_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
        tf.data.experimental.AUTOTUNE)
    val_ds = val_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Code to define the model
    model = tf.keras.models.Sequential([
        Conv2D(32,(3,3),activation='relu',input_shape = (30,30,3)),
        MaxPooling2D(2,2),
        Conv2D(64,(3,3),activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(64,(3,3),activation='relu'),
        MaxPooling2D(2,2), # 이미지 자체가 너무 작기때문에 맥스풀링 너무 많이 x
        Flatten(),
        Dropout(0.25),
        Dense(128,activation='relu'),
        tf.keras.layers.Dense(43, activation=tf.nn.softmax)
    ])

    model.compile( optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['acc']
    )

    checkpoint_path = 'tmp_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 save_best_only = True,
                                 monitor = 'val_loss',
                                 verbose=1)

    model.fit(
        train_ds,
        validation_data = val_ds,
        epochs=10,
        callbacks=[checkpoint]
    )
    model.load_weights(checkpoint_path)
    

    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

fashion mnist 사용

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint


def solution_model():
    fashion_mnist = tf.keras.datasets.fashion_mnist
    

    (x_train,y_train),(x_valid,y_valid) = fashion_mnist.load_data()

    x_train = x_train/255
    x_valid = x_valid/255



    model = Sequential([
        Flatten(),
        Dropout(0.25),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(256, activation='relu'),
        Dropout(0.25),
        Dense(64, activation='relu'),
        Dense(32, activation = 'relu'),
    # Classification을 위한 Softmax 
        Dense(10, activation='softmax'),
        ])
    
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

    checkpoint_path = "my_checkpoint.ckpt"
    checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                                 save_weights_only=True, 
                                save_best_only=True, 
                                monitor='val_loss', 
                                verbose=1)
    
    model.fit(x_train, y_train,
                    validation_data=(x_valid, y_valid),
                    epochs=20,
                    callbacks=[checkpoint],
                   )
    

    model.load_weights(checkpoint_path)


    
    return model


if __name__ == '__main__':
    model = solution_model()
    model.save("TF2-fashion-mnist_7.h5")

자연어 처리 모델

In [ ]:

import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
    urllib.request.urlretrieve(url, 'sarcasm.json')

    with open('sarcasm.json') as f:
        datas = json.load(f)
        
    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    training_size = 20000


    sentences = []
    labels = []


    for data in datas:
      sentences.append(data['headline'])
      labels.append(data['is_sarcastic'])




    train_sentences = sentences[:training_size]
    train_labels = labels[:training_size]

    validation_sentences = sentences[training_size:]
    validation_labels = labels[training_size:]

    

    tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')

    tokenizer.fit_on_texts(train_sentences)
    for key, value in tokenizer.word_index.items():
        print('{}  \t======>\t {}'.format(key, value))
        if value == 25:
          break
    


    train_sequences = tokenizer.texts_to_sequences(train_sentences)
    validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

    train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)
    validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    train_labels = np.array(train_labels)
    validation_labels = np.array(validation_labels)


    model = tf.keras.Sequential([
        
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(64)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.summary()

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    checkpoint_path = 'my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss',
                             verbose=1)
    

    epochs=10
    history = model.fit(train_padded, train_labels, 
                        validation_data=(validation_padded, validation_labels),
                        callbacks=[checkpoint],
                        epochs=epochs)
    
    model.load_weights(checkpoint_path)




    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("TF4-sarcasm.h5")

뉴럴네트워크

In [ ]:

import urllib
import os
import zipfile
import pandas as pd
import tensorflow as tf


import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()


# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size= (n_past + n_future), # YOUR CODE HERE,
                   shift = shift, drop_remainder = True)
    # This line converts the windowed dataset into a tensorflow dataset.
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(
        lambda w: (w[:n_past], w[n_past:])  ######과거 24개는 x 데이터, 미래 24개는 y데이터
    )
    return ds.batch(batch_size).prefetch(1)

def solution_model():
    # Downloads and extracts the dataset to the directory that
    # contains this file.
    download_and_extract_data()
    # Reads the dataset from the csv.
    df = pd.read_csv('household_power_consumption.csv', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)
    # Number of features in the dataset. We use all features as predictors to
    # predict all features at future time steps.
    
    N_FEATURES = len(df.columns)
    # Normalizes the data
    data = df.values
    split_time = int(len(data) * 0.8)
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))
    # Splits the data into training and validation sets.
    x_train = data[:split_time]
    x_valid = data[split_time:]
    # DO NOT CHANGE 'BATCH_SIZE' IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS
    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
    N_PAST = 24  # DO NOT CHANGE THIS
    # Number of future time steps which are to be predicted.
    N_FUTURE = 24  # DO NOT CHANGE THIS
    # By how many positions the window slides to create a new window
    # of observations.
    SHIFT = 1  # DO NOT CHANGE THIS
    # Code to create windowed train and validation datasets.
    # Complete the code in windowed_dataset.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    # Code to define your model.
    model = tf.keras.models.Sequential([
        Conv1D(filters=32,   # 데이터 사이즈가 선스팟 데이터보다 커서 오래 걸림, filter도 32, lstm도 32로 줌
                kernel_size=3,
                padding="causal",
                activation="relu",
                input_shape=[N_PAST, 7],   #24개 * 7개가 들어가고  아웃풋이 24*7 데이터가 예측되어서 나옴
                ),
        Bidirectional(LSTM(32, return_sequences=True)),
        Dense(32, activation="relu"),
        Dense(16, activation="relu"),
        Dense(N_FEATURES)
    ])

    checkpoint_path='model/my_checkpoint.ckpt'

    checkpoint = ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss', #  컴파일 할 때 mae로 설정할거기대문에 val_loss ok
                             verbose=1,
                             )
    # learning_rate=0.0005, Adam 옵치마이저
    optimizer =  tf.keras.optimizers.Adam(learning_rate=0.0005) # sgd 썼더니 학습속도 느려서 adam 씀

    model.compile(loss='mae', #문제에서 mae 기준으로 0.55 어쩌고 적혀잇음
              optimizer=optimizer,
              metrics=["mae"]
              )
    model.fit(train_set, 
        validation_data=(valid_set), 
        epochs=20, 
        callbacks=[checkpoint], 
        )
    
    model.load_weights(checkpoint_path)
    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("TF5_HEPC_1.h5")